# Calculation of Spectrum Averaged Cross Sections

In this notebook we perform a fit with GMAP and then compute posterior estimates of Spectrum Averaged Cross Sections (SACS) and their uncertainties using the sandwich formula. We will make two GMAP evaluations: One will be in legacy mode with the PPP bug and SACS Jacobian bug present and the other one without these bugs. Both calculations will be done with the PPP correction enabled (i.e., percentual experimental uncertainties are interpreted as being relative to the fitted cross sections).

First we load required packages:

In [1]:
import sys
sys.path.append('../..')  # so that gmapy can be found
import numpy as np
import pandas as pd
from gmapy.legacy.legacy_gmap import run_gmap

We use a database file from the test data. This file is also the basis for the neutron standards evaluation 2017.

In [2]:
dbfile = '../../tests/testdata/data-2017-07-26.gma'

The next lines perform the calculations, the first one in legacy mode and the other one in modern mode.

In [3]:
%%capture 
res1 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=False, fix_sacs_jacobian=False, legacy_output=False)

res2 = run_gmap(dbfile, dbtype='legacy', num_iter=3, correct_ppp=True,
                fix_ppp_bug=True, fix_sacs_jacobian=True, legacy_output=False)

*Note*: The `%%capture` statement is there to discard the output of this cell, which would otherwise produce a warning about a division by zero caused by some experimental dataset.

## Construction of a synthetic experiment table 

SACS values and their uncertainties are not estimated by the GMAP functions above but some SACS measurements are nevertheless used as constraints during inference. To obtain posterior estimates and uncertainties of the SACS values, we need to construct a synthetic experiment table and propagate the posterior estimates of the cross sections manually to the SACS values.

To construct a synthetic experiment table, let us remind ourselves of the structure of an experiment table by inspecting the one used in the fits above. We select only SACS measurements to obtain a concise table.

In [5]:
sacs_sel = (res1['table'].NODE.str.match('MT:6-') & res1['table'].NODE.str.match('exp_'))
sacs_exptable = res1['table'][res1['table'].REAC.str.match('MT:6-')].reset_index()
sacs_exptable

,index,NODE,REAC,ENERGY,PRIOR,UNC,DESCR,DATA,DB_IDX,DS_IDX,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC
0,2364,exp_565,MT:6-R1:8,0.15,NaN,NaN,NaN,1.215,55.0,2.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN
1,2365,exp_641,MT:6-R1:9,1.50,NaN,NaN,NaN,1.790,55.0,3.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN
2,2367,exp_575,MT:6-R1:8,1.20,NaN,NaN,NaN,1.254,55.0,5.0,V.M.ADAMOV ET AL.,INDC(CCP)-180L,0.019668,1.224776,NaN,NaN
3,2368,exp_576,MT:6-R1:8,1.20,NaN,NaN,NaN,1.216,55.0,6.0,H.T.HEATON ET AL.,MEMO J.GRUNDLE,0.019944,1.224776,NaN,NaN
4,2369,exp_674,MT:6-R1:9,1.20,NaN,NaN,NaN,1.824,55.0,7.0,H.T.HEATON ET AL.,ANS 44-533 DLE,0.030412,1.805102,NaN,NaN
5,2370,exp_517,MT:6-R1:8,1.50,NaN,NaN,NaN,1.234,55.0,8.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.018052,1.224776,NaN,NaN
6,2371,exp_614,MT:6-R1:9,1.50,NaN,NaN,NaN,1.844,55.0,9.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.026402,1.805102,NaN,NaN


The relevant columns that must be present for a mapping are `NODE`, `REAC`, and `ENERGY`. The `DATA` column contains the SACS measurements and is only used for fitting. Let's augment this table by the reaction string in human-readable form.

In [6]:
posttable1 = res1['table']
posttable2 = res2['table']
reac_descr = [posttable1[posttable1.REAC == p].DESCR.iloc[0]
              for p in sacs_exptable['REAC'].str.replace('MT:6-','MT:1-').tolist()]
sacs_exptable['DESCR'] = reac_descr
sacs_exptable

,index,NODE,REAC,ENERGY,PRIOR,UNC,DESCR,DATA,DB_IDX,DS_IDX,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC
0,2364,exp_565,MT:6-R1:8,0.15,NaN,NaN,"U5(n,f)",1.215,55.0,2.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN
1,2365,exp_641,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",1.790,55.0,3.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN
2,2367,exp_575,MT:6-R1:8,1.20,NaN,NaN,"U5(n,f)",1.254,55.0,5.0,V.M.ADAMOV ET AL.,INDC(CCP)-180L,0.019668,1.224776,NaN,NaN
3,2368,exp_576,MT:6-R1:8,1.20,NaN,NaN,"U5(n,f)",1.216,55.0,6.0,H.T.HEATON ET AL.,MEMO J.GRUNDLE,0.019944,1.224776,NaN,NaN
4,2369,exp_674,MT:6-R1:9,1.20,NaN,NaN,"PU9(n,f)",1.824,55.0,7.0,H.T.HEATON ET AL.,ANS 44-533 DLE,0.030412,1.805102,NaN,NaN
5,2370,exp_517,MT:6-R1:8,1.50,NaN,NaN,"U5(n,f)",1.234,55.0,8.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.018052,1.224776,NaN,NaN
6,2371,exp_614,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",1.844,55.0,9.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.026402,1.805102,NaN,NaN


This table shows that several measurements of the same SACS quantity have been made. For the prediction, we keep the unique reactions only.

In [7]:
syntable = sacs_exptable.groupby('REAC').first().reset_index().drop(['index', 'DB_IDX', 'DS_IDX', 'DATA'], axis=1)
syntable1 = pd.concat([res1['table'][~res1['table'].NODE.str.match('exp_')], syntable], ignore_index=True)
syntable2 = pd.concat([res2['table'][~res2['table'].NODE.str.match('exp_')], syntable], ignore_index=True)

## Calculation of posterior SACS values and uncertainties

Now let's propagate the estimates of the basic cross sections to obtain the SACS values.

In [8]:
from gmapy.mappings.compound_map import CompoundMap
compmap = CompoundMap(fix_sacs_jacobian=True)
predvals1 = compmap.propagate(syntable1.POST.to_numpy(), syntable1)
predvals2 = compmap.propagate(syntable2.POST.to_numpy(), syntable2)
syntable1['PRED1'] = predvals1
syntable2['PRED2'] = predvals2
syntable1[syntable1.REAC.str.match('MT:6-')]

,NODE,REAC,ENERGY,PRIOR,UNC,DESCR,DATA,DB_IDX,DS_IDX,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC,PRED1
1227,exp_565,MT:6-R1:8,0.15,NaN,NaN,"U5(n,f)",NaN,NaN,NaN,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN,1.224758
1228,exp_641,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",NaN,NaN,NaN,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN,1.800731


In [9]:
syntable2[syntable2.REAC.str.match('MT:6-')]

,NODE,REAC,ENERGY,PRIOR,UNC,DESCR,DATA,DB_IDX,DS_IDX,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC,PRED2
1227,exp_565,MT:6-R1:8,0.15,NaN,NaN,"U5(n,f)",NaN,NaN,NaN,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN,1.225180
1228,exp_641,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",NaN,NaN,NaN,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN,1.801952


We also want posterior uncertainties for the SACS values. We obtain them by relying on the sensitivity matrix  and the posterior covariance matrix. 

In [10]:
sacs_sel = np.ravel(np.where((syntable1.NODE.str.match('exp_'))))
prior_sel = np.ravel(np.where((~syntable1.NODE.str.match('exp_')) &
                              (~syntable1.NODE.str.fullmatch('fis'))))
S1 = compmap.jacobian(syntable1.POST.to_numpy(), syntable1)
S1 = S1[np.ix_(sacs_sel,prior_sel)]
postcov1 = res1['postcov']
sacs_unc1 = np.sqrt(np.diag(np.array(S1 @ postcov1 @ S1.T)))


S2 = compmap.jacobian(syntable2.POST.to_numpy(), syntable2)
S2 = S2[np.ix_(sacs_sel,prior_sel)]
postcov2 = res2['postcov']
sacs_unc2 = np.sqrt(np.diag(np.array(S2 @ postcov2 @ S2.T)))
syntable['UNC1'] = sacs_unc1
syntable['UNC2'] = sacs_unc2
syntable

,REAC,NODE,ENERGY,PRIOR,UNC,DESCR,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC,UNC1,UNC2
0,MT:6-R1:8,exp_565,0.15,NaN,NaN,"U5(n,f)",M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN,0.003466,0.003468
1,MT:6-R1:9,exp_641,1.50,NaN,NaN,"PU9(n,f)",M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN,0.006048,0.006047


Finally, we compare this to the experiment table:

In [11]:
sacs_exptable.sort_values('REAC')

,index,NODE,REAC,ENERGY,PRIOR,UNC,DESCR,DATA,DB_IDX,DS_IDX,AUTHOR,PUBREF,DATAUNC,POST,POSTUNC,RELPOSTUNC
0,2364,exp_565,MT:6-R1:8,0.15,NaN,NaN,"U5(n,f)",1.215,55.0,2.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.024572,1.224776,NaN,NaN
2,2367,exp_575,MT:6-R1:8,1.20,NaN,NaN,"U5(n,f)",1.254,55.0,5.0,V.M.ADAMOV ET AL.,INDC(CCP)-180L,0.019668,1.224776,NaN,NaN
3,2368,exp_576,MT:6-R1:8,1.20,NaN,NaN,"U5(n,f)",1.216,55.0,6.0,H.T.HEATON ET AL.,MEMO J.GRUNDLE,0.019944,1.224776,NaN,NaN
5,2370,exp_517,MT:6-R1:8,1.50,NaN,NaN,"U5(n,f)",1.234,55.0,8.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.018052,1.224776,NaN,NaN
1,2365,exp_641,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",1.790,55.0,3.0,M.C.DAVIS ET AL.,"ANE5,583(1978)",0.044173,1.805102,NaN,NaN
4,2369,exp_674,MT:6-R1:9,1.20,NaN,NaN,"PU9(n,f)",1.824,55.0,7.0,H.T.HEATON ET AL.,ANS 44-533 DLE,0.030412,1.805102,NaN,NaN
6,2371,exp_614,MT:6-R1:9,1.50,NaN,NaN,"PU9(n,f)",1.844,55.0,9.0,I.G.SCHROEDER ET AL.,84GEEL+PC,0.026402,1.805102,NaN,NaN
